In [30]:
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def max_length_text(df):
    max_length = 0
    for i in range(df.shape[0]):
        length = np.size(word_tokenize(df.at[i, 'text']))
        if length > max_length: max_length = length
    return max_length

In [4]:
def text_normalization(data):
    data['text'] = data['text'].apply(lambda x: x.lower())
    data['text'] = data['text'].apply((lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x)))

In [5]:
def remove_stop_words(data, language, get_tokenize):
    stopwords = nltk.corpus.stopwords.words(language)
    if get_tokenize:
        for i in range(data.shape[0]):
            data.at[i, 'text'] = [word for word in nltk.word_tokenize(data.at[i, 'text']) if word not in stopwords]
    else:
        for i in range(data.shape[0]):
            data.at[i, 'text'] = [word for word in nltk.word_tokenize(data.at[i, 'text']) if word not in stopwords]
            data.at[i, 'text'] = ' '.join(data.at[i, 'text'])

In [6]:
def apply_stemming(data, language):
    stemmer = SnowballStemmer(language)
    for i in range(data.shape[0]):
         data.at[i, 'text'] = (' '.join([stemmer.stem(word) for word in data.at[i, 'text'].split()]))

#### get_matrix representation | BoW and Tf-idf for Classic ML

In [7]:
def get_matrix(data, representation, vocabulary_length, stemming, remove_stopwords, language):

    df = data.copy(deep = True)
    
    text_normalization(df) # Text normalization
    
    # Stop_words
    if remove_stopwords:
        remove_stop_words(df, language, False)
    
    # Stemming
    if stemming:
        apply_stemming(df, language)
    
    # Word representation
    if representation == 'BoW':
        count_vectorizer = CountVectorizer(max_df = 0.9, max_features = vocabulary_length, min_df = 0)
        #count_vectorizer = CountVectorizer(max_features = vocabulary_length)
        matrix = count_vectorizer.fit_transform(df.text)
        
    elif representation == 'tf-idf':
        tfidf_vectorizer = TfidfVectorizer(max_df = 0.9, max_features = vocabulary_length, min_df = 0, use_idf = True)
        #tfidf_vectorizer = TfidfVectorizer(max_features = vocabulary_length, use_idf=True)
        matrix = tfidf_vectorizer.fit_transform(df.text)
    
    return matrix, df

#### preprocessing for RNN - LSTM

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
def get_input_RNN(data, stemming, remove_stopwords, vocabulary_length, max_length_sequence, language):
    
    df = data.copy(deep = True)
    
    text_normalization(df) # Text normalization
    
    # Stemming
    if stemming:
        apply_stemming(df, language)
    
    # Stop_words
    if remove_stopwords:
        remove_stop_words(df, language, True)
        
    # Tokenizer
    tokenizer = Tokenizer(num_words = vocabulary_length)
    tokenizer.fit_on_texts(df.text)
    X = tokenizer.texts_to_sequences(df.text)
    
    # Padding
    X = pad_sequences(X, maxlen = max_length_sequence, padding = 'post', truncating = 'post')
    
    return X, df

#### Tests

In [33]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
...,...,...
2566,"Recuperamos la historia de Aleixandra, la jove...",0
2567,"Reproches, tensión y sinceridad: la comida en ...",0
2568,"RT @ElMundoOpinion: ""PSOE, PP, Ciudadanos y Vo...",0
2569,Rusia cita al embajador español por unas decla...,0


In [31]:
max_length_text(dataset)

2865

In [15]:
print(dataset.at[0, 'text'])

RAE INCLUIRÁ LA PALABRA "LADY" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE "MUJER PROBLEMÁTICA"
España.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término "Lady" como una nueva palabra en el diccionario del idioma español.
Darío señaló que "Lady" servirá para definir a una "mujer problemática" o a una "mujer que causa problemas", y mencionó que esta palabra será una de las pocas que también se utilizan en el idioma inglés pero que en castellano tiene un significado diferente:
"Son contadas las palabras del idioma inglés que se utilizan en el español pero que tienen otro significado. Con la globalización las personas han comenzado a adoptar términos anglosajones pero los utilizan con su significado real, sin embargo en este caso la expresión Lady no significará lo mismo que en su idioma original ("dama" en inglés) sino que se usará para definir a una mujer que es problemáti

In [41]:
matrix, df = get_input_RNN(dataset, True, True, 40000, 2900, language = 'spanish')
print(df.at[0, 'text'])
print(df.shape)

['rae', 'inclu', 'palabr', 'lady', 'diccionari', 'idiom', 'español', 'com', 'definicion', 'muj', 'problemat', 'españ', 'president', 'real', 'academi', 'español', 'rae', 'dari', 'villanuev', 'inform', 'conferent', 'prens', 'part', 'proxim', 'mes', 'inclu', 'termin', 'lady', 'com', 'nuev', 'palabr', 'diccionari', 'idiom', 'español', 'dari', 'señal', 'lady', 'serv', 'par', 'defin', 'muj', 'problemat', 'muj', 'caus', 'problem', 'mencion', 'palabr', 'ser', 'poc', 'tambien', 'utiliz', 'idiom', 'ingles', 'per', 'castellan', 'tien', 'signific', 'diferent', 'cont', 'palabr', 'idiom', 'ingles', 'utiliz', 'español', 'per', 'tien', 'signific', 'globaliz', 'person', 'comenz', 'adopt', 'termin', 'anglosajon', 'per', 'utiliz', 'signific', 'real', 'embarg', 'cas', 'expresion', 'lady', 'signific', 'mism', 'idiom', 'original', 'dam', 'ingles', 'usar', 'par', 'defin', 'muj', 'problemat', 'acostumbr', 'caus', 'problem', 'alborot', 'gent', 'podr', 'dec', 'lady', 'femin', 'caus', 'algun', 'escandal', 'agres

In [45]:
print(len(matrix))
print(matrix[0])
print(len(matrix[0]))

2571
[1538  235  422 ...    0    0    0]
2900


In [46]:
print(matrix[1])
print(len(matrix[1]))

[ 422 5516  368 ...    0    0    0]
2900
